# Üretici Ağlar

Tekrarlayan Sinir Ağları (RNN'ler) ve Uzun Kısa Süreli Bellek Hücreleri (LSTM'ler) ile Gated Recurrent Units (GRU'lar) gibi kapılı hücre varyantları, dil modelleme için bir mekanizma sağladı, yani kelime sıralamasını öğrenebilir ve bir dizideki bir sonraki kelime için tahminlerde bulunabilirler. Bu, RNN'leri **üretici görevler** için kullanmamıza olanak tanır, örneğin sıradan metin üretimi, makine çevirisi ve hatta görüntü açıklaması.

Önceki birimde tartıştığımız RNN mimarisinde, her RNN birimi bir sonraki gizli durumu çıktı olarak üretiyordu. Ancak, her tekrarlayan birime başka bir çıktı da ekleyebiliriz, bu da bize bir **dizi** (orijinal diziyle aynı uzunlukta) üretme imkanı sağlar. Dahası, her adımda bir girdi kabul etmeyen ve sadece bir başlangıç durum vektörü alarak bir çıktı dizisi üreten RNN birimlerini de kullanabiliriz.

Bu not defterinde, metin üretmemize yardımcı olan basit üretici modeller üzerine odaklanacağız. Basitlik adına, harf harf metin üreten bir **karakter düzeyinde ağ** oluşturalım. Eğitim sırasında, bir metin korpusunu alıp harf dizilerine bölmemiz gerekecek.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Karakter Kelime Dağarcığı Oluşturma

Karakter seviyesinde bir üretici ağ oluşturmak için metni kelimeler yerine bireysel karakterlere ayırmamız gerekiyor. Daha önce kullandığımız `TextVectorization` katmanı bunu yapamaz, bu yüzden iki seçeneğimiz var:

* Metni manuel olarak yükleyip, [bu resmi Keras örneğinde](https://keras.io/examples/generative/lstm_character_level_text_generation/) olduğu gibi 'elle' tokenizasyon yapmak.
* Karakter seviyesinde tokenizasyon için `Tokenizer` sınıfını kullanmak.

Biz ikinci seçeneği tercih edeceğiz. `Tokenizer`, kelimelere tokenizasyon yapmak için de kullanılabilir, bu nedenle karakter seviyesinden kelime seviyesine tokenizasyona geçiş oldukça kolay olmalıdır.

Karakter seviyesinde tokenizasyon yapmak için `char_level=True` parametresini geçmemiz gerekiyor:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Dizinin **sonunu** belirtmek için `<eos>` olarak adlandıracağımız özel bir token kullanmak istiyoruz. Hadi bunu kelime dağarcığına manuel olarak ekleyelim:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Şimdi, metni sayı dizilerine dönüştürmek için şunları kullanabiliriz:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Başlıklar Üretmek için Bir Generative RNN Eğitmek

RNN'i haber başlıkları üretmek için şu şekilde eğiteceğiz. Her adımda bir başlık alacağız, bu başlık bir RNN'e verilecek ve her bir giriş karakteri için ağdan bir sonraki çıkış karakterini üretmesini isteyeceğiz:

!['HELLO' kelimesinin bir RNN tarafından nasıl üretildiğini gösteren bir örnek görüntü.](../../../../../translated_images/tr/rnn-generate.56c54afb52f9781d.webp)

Dizimizin son karakteri için ağdan `<eos>` tokenini üretmesini isteyeceğiz.

Burada kullandığımız generative RNN'in temel farkı, RNN'in her adımından bir çıktı alacak olmamızdır, sadece son hücreden değil. Bu, RNN hücresine `return_sequences` parametresini belirterek gerçekleştirilebilir.

Bu nedenle, eğitim sırasında, ağa bir giriş olarak belirli bir uzunlukta kodlanmış karakter dizisi verilecek ve bir çıkış olarak aynı uzunlukta, ancak bir eleman kaydırılmış ve `<eos>` ile sonlandırılmış bir dizi alınacaktır. Minibatch, bu tür birkaç diziden oluşacak ve tüm dizileri hizalamak için **padding** kullanmamız gerekecek.

Şimdi, veri setini bizim için dönüştürecek fonksiyonlar oluşturalım. Dizileri minibatch seviyesinde doldurmak istediğimiz için, önce `.batch()` çağrısı yaparak veri setini gruplandıracağız ve ardından dönüşüm yapmak için `map` kullanacağız. Bu nedenle, dönüşüm fonksiyonu bir minibatch'in tamamını parametre olarak alacak:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Burada yaptığımız birkaç önemli şey:
* Öncelikle, string tensöründen gerçek metni çıkarıyoruz
* `text_to_sequences`, string listesini bir tamsayı tensörleri listesine dönüştürür
* `pad_sequences`, bu tensörleri maksimum uzunluklarına kadar doldurur
* Son olarak, tüm karakterleri one-hot encode ediyoruz, ayrıca kaydırma ve `<eos>` ekleme işlemini yapıyoruz. Karakterleri neden one-hot encode ettiğimizi yakında göreceğiz

Ancak, bu fonksiyon **Pythonic** bir yapıya sahiptir, yani Tensorflow hesaplama grafiğine otomatik olarak çevrilemez. Bu fonksiyonu doğrudan `Dataset.map` fonksiyonunda kullanmaya çalışırsak hata alırız. Bu Pythonic çağrıyı `py_function` sarmalayıcısını kullanarak çevrelememiz gerekiyor:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Not**: Pythonic ve Tensorflow dönüşüm fonksiyonları arasındaki farkı anlamak biraz karmaşık görünebilir ve neden veri setini `fit` fonksiyonuna geçmeden önce standart Python fonksiyonlarıyla dönüştürmediğimizi sorguluyor olabilirsiniz. Bu kesinlikle yapılabilir, ancak `Dataset.map` kullanmanın büyük bir avantajı vardır: veri dönüşüm hattı Tensorflow hesaplama grafiği kullanılarak yürütülür, bu da GPU hesaplamalarından faydalanır ve CPU/GPU arasında veri geçişi ihtiyacını en aza indirir.

Şimdi jeneratör ağımızı oluşturabilir ve eğitime başlayabiliriz. Bu ağ, önceki birimde tartıştığımız herhangi bir tekrarlayan hücreye (basit, LSTM veya GRU) dayanabilir. Örneğimizde LSTM kullanacağız.

Ağ karakterleri girdi olarak aldığı ve kelime dağarcığı boyutu oldukça küçük olduğu için, gömme katmanına ihtiyacımız yoktur; tekil kodlanmış girdi doğrudan LSTM hücresine gidebilir. Çıkış katmanı, LSTM çıktısını tekil kodlanmış token numaralarına dönüştürecek bir `Dense` sınıflandırıcı olacaktır.

Ek olarak, değişken uzunlukta dizilerle çalıştığımız için, dizinin doldurulmuş kısmını görmezden gelecek bir maske oluşturmak için `Masking` katmanını kullanabiliriz. Bu kesinlikle gerekli değildir, çünkü `<eos>` token'ını aşan her şeyle çok fazla ilgilenmiyoruz, ancak bu katman türüyle biraz deneyim kazanmak adına kullanacağız. `input_shape` `(None, vocab_size)` olacaktır, burada `None` değişken uzunlukta diziyi belirtir ve çıkış şekli de `(None, vocab_size)` olacaktır, `summary` çıktısından da görebileceğiniz gibi:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Çıktı oluşturma

Modeli eğittikten sonra, şimdi onu kullanarak bazı çıktılar üretmek istiyoruz. Öncelikle, bir dizi token numarasıyla temsil edilen metni çözmek için bir yönteme ihtiyacımız var. Bunun için `tokenizer.sequences_to_texts` fonksiyonunu kullanabiliriz; ancak bu fonksiyon karakter düzeyinde tokenizasyonla iyi çalışmaz. Bu nedenle, tokenizer'dan (adı `word_index` olan) bir token sözlüğü alacağız, ters bir harita oluşturacağız ve kendi çözümleme fonksiyonumuzu yazacağız:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Şimdi, üretim yapacağız. Öncelikle bir dize olan `start` ile başlayacağız, bunu bir diziye `inp` olarak kodlayacağız ve ardından her adımda ağımızı çağırarak bir sonraki karakteri tahmin edeceğiz.

Ağdan gelen çıktı `out`, her bir tokenin olasılıklarını temsil eden `vocab_size` öğeden oluşan bir vektördür ve en olası token numarasını `argmax` kullanarak bulabiliriz. Daha sonra bu karakteri üretilen token listesinin sonuna ekleriz ve üretime devam ederiz. Bir karakter üretme işlemi, gerekli karakter sayısını üretmek için `size` kez tekrarlanır ve `eos_token` ile karşılaşıldığında erken sonlandırılır.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Eğitim sırasında çıktı örnekleme

*Doğruluk* gibi faydalı metriklere sahip olmadığımız için, modelimizin daha iyi hale geldiğini görmenin tek yolu, eğitim sırasında üretilen dizelerden **örnekleme** yapmaktır. Bunu yapmak için, `fit` fonksiyonuna geçirebileceğimiz ve eğitim sırasında periyodik olarak çağrılacak olan **geri çağrım fonksiyonlarını** (callbacks) kullanacağız.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Bu örnek zaten oldukça iyi bir metin üretiyor, ancak birkaç şekilde daha da geliştirilebilir:

* **Daha fazla metin**. Görevimiz için yalnızca başlıkları kullandık, ancak tam metinle denemeler yapmak isteyebilirsiniz. RNN'lerin uzun dizileri işleme konusunda pek iyi olmadığını unutmayın, bu yüzden ya bunları daha kısa cümlelere bölmek ya da her zaman önceden tanımlanmış bir `num_chars` değeri (örneğin, 256) ile sabit bir dizi uzunluğunda eğitmek mantıklı olabilir. Yukarıdaki örneği böyle bir mimariye dönüştürmeyi deneyebilir ve [resmi Keras eğitimi](https://keras.io/examples/generative/lstm_character_level_text_generation/) ile ilham alabilirsiniz.

* **Çok katmanlı LSTM**. 2 veya 3 katmanlı LSTM hücrelerini denemek mantıklı olabilir. Önceki birimde belirttiğimiz gibi, her LSTM katmanı metinden belirli desenler çıkarır ve karakter düzeyinde bir üretici durumunda, daha düşük LSTM seviyesinin heceleri çıkarmaktan, daha yüksek seviyelerin ise kelimeler ve kelime kombinasyonlarından sorumlu olmasını bekleyebiliriz. Bu, LSTM yapıcısına katman sayısı parametresi geçirerek basitçe uygulanabilir.

* **GRU birimleri** ile denemeler yapmak ve hangilerinin daha iyi performans gösterdiğini görmek isteyebilirsiniz, ayrıca **farklı gizli katman boyutları** ile de denemeler yapabilirsiniz. Çok büyük bir gizli katman aşırı öğrenmeye yol açabilir (örneğin, ağ tam metni öğrenir) ve daha küçük bir boyut iyi bir sonuç üretmeyebilir.


## Yumuşak metin üretimi ve sıcaklık

`generate` fonksiyonunun önceki tanımında, oluşturulan metindeki bir sonraki karakter olarak her zaman en yüksek olasılığa sahip karakteri seçiyorduk. Bu durum, metnin sık sık aynı karakter dizileri arasında "dönmesine" neden oluyordu, tıpkı şu örnekte olduğu gibi:
```
today of the second the company and a second the company ...
```

Ancak, bir sonraki karakter için olasılık dağılımına baktığımızda, en yüksek birkaç olasılık arasındaki farkın çok büyük olmayabileceğini görebiliriz. Örneğin, bir karakterin olasılığı 0.2, diğerinin ise 0.19 olabilir, vb. Örneğin, '*play*' dizisindeki bir sonraki karakteri ararken, bir sonraki karakterin boşluk ya da **e** (örneğin *player* kelimesindeki gibi) olması eşit derecede olasıdır.

Bu durum bizi şu sonuca götürür: Her zaman en yüksek olasılığa sahip karakteri seçmek "adil" olmayabilir, çünkü ikinci en yüksek olasılığa sahip karakteri seçmek de anlamlı bir metne yol açabilir. Bu nedenle, ağın çıktısının verdiği olasılık dağılımından karakterleri **örneklemek** daha akıllıca bir yaklaşımdır.

Bu örnekleme, **multinomial dağılım** olarak adlandırılan bir yöntemi uygulayan `np.multinomial` fonksiyonu kullanılarak yapılabilir. Bu **yumuşak** metin üretimini gerçekleştiren bir fonksiyon aşağıda tanımlanmıştır:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

En yüksek olasılığa ne kadar sıkı bağlı kalmamız gerektiğini belirten **temperature** adlı bir parametre daha tanıttık. Eğer temperature 1.0 ise, adil bir multinomial örnekleme yaparız ve temperature sonsuza yaklaştığında - tüm olasılıklar eşit hale gelir ve bir sonraki karakteri rastgele seçeriz. Aşağıdaki örnekte, temperature değerini çok fazla artırdığımızda metnin anlamsız hale geldiğini ve 0'a yaklaştığında "döngüsel" şekilde zor üretilmiş bir metne benzediğini gözlemleyebiliriz.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlıklar içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
